In [ ]:
!pip install praw
import praw
from praw.models import MoreComments
!pip install textblob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
!pip install gensim
from gensim import matutils, models
import scipy.sparse
from textblob import TextBlob
import pandas as pd
import nltk
import re 

In [ ]:
#obtaining access to Reddit API
reddit = praw.Reddit(client_id = 'wlWQaMEIZXxrgQ',
                     client_secret = 'sC-GfU6JWXA-ikud7b2G-f_kFZ8',
                     user_agent='BHI Scraper')

is there a way to scrape every post that posters in r/parenting have made?


In [ ]:
#obtaining current posts
posts = reddit.subreddit('Parenting').top(limit=10)

In [4]:
#testing out the posts
for post in posts:
    print(post.title)

[Update]35 year old Dad diagnosed with a terminal illness. How do I tell my 4 year old little girl?
Gave birth in our driveway this morning
I'm dealing with depression and I always try to hide it from my 3 year old son. Today I couldn't hide it and I am so proud of his reaction.
I’ve never been so proud of my 16 year-old son
I got to witness the beginning of a generational pattern starting to break.
When a 33 year old dad spends a day in kindergarten: top 10 observations
Got checked by my toddler today
My 4 year old gave me an ugly reality check
So we just had a kid (a bit tmi)
After 6 years of being the "working" parent, I've accepted the role of stay at home dad.


In [5]:
#creating data table of posts and other information from parenting and daddit
parenting_posts = []
num_posts = 5
r_parenting = reddit.subreddit('Parenting')
r_daddit = reddit.subreddit('daddit')

for post in r_parenting.top(limit = num_posts):
    parenting_posts.append([post.id, post.title, post.score, post.subreddit, 
                            post.url, post.num_comments, post.selftext])
    
for post in r_daddit.top(limit = num_posts):
    parenting_posts.append([post.id, post.title, post.score, post.subreddit, 
                            post.url, post.num_comments, post.selftext])
    
parenting_posts = pd.DataFrame(parenting_posts, columns = ['id', 'title', 'score', 'subreddit', 'url', 'num_comments', 'text'])
parenting_posts.head()

,id,title,score,subreddit,url,num_comments,text
0,76x0q4,[Update]35 year old Dad diagnosed with a termi...,9464,Parenting,https://www.reddit.com/r/Parenting/comments/76...,304,I want you all to know that I had no idea Scot...
1,gh7f74,Gave birth in our driveway this morning,6235,Parenting,https://www.reddit.com/r/Parenting/comments/gh...,301,Just a sigh of relief via text! We were all pr...
2,bmjbfv,I'm dealing with depression and I always try t...,5634,Parenting,https://www.reddit.com/r/Parenting/comments/bm...,258,Had a rough morning this morning. Dealing with...
3,e03s5e,I’ve never been so proud of my 16 year-old son,5307,Parenting,https://www.reddit.com/r/Parenting/comments/e0...,190,"About 10 years ago, my first wife and my daugh..."
4,gdns9x,I got to witness the beginning of a generation...,5135,Parenting,https://www.reddit.com/r/Parenting/comments/gd...,161,My 4 yr old daughter just called out my toxic ...


In [135]:
#obtaining the comments from posts and adding it to the table
parenting_comments = []
post_id = parenting_posts['id']
for key in range(len(post_id)):
    parenting_comments.append([comments.body for comments in reddit.submission(id = post_id[key]).comments if not isinstance(comments,MoreComments)])
parenting_posts['comments'] = parenting_comments
parenting_posts.head()

,id,title,score,subreddit,url,num_comments,text,comments
0,76x0q4,[Update]35 year old Dad diagnosed with a termi...,9463,Parenting,https://www.reddit.com/r/Parenting/comments/76...,304,I want you all to know that I had no idea Scot...,[This is beautiful. What an incredible father....
1,gh7f74,Gave birth in our driveway this morning,6236,Parenting,https://www.reddit.com/r/Parenting/comments/gh...,301,Just a sigh of relief via text! We were all pr...,"[Holy moly, what a way to celebrate Mother's D..."
2,bmjbfv,I'm dealing with depression and I always try t...,5632,Parenting,https://www.reddit.com/r/Parenting/comments/bm...,258,Had a rough morning this morning. Dealing with...,"[You know, he learned to be that kind, gentle,..."
3,e03s5e,I’ve never been so proud of my 16 year-old son,5306,Parenting,https://www.reddit.com/r/Parenting/comments/e0...,190,"About 10 years ago, my first wife and my daugh...","[Yeah, I'm crying now too., Beautiful. Kids ar..."
4,gdns9x,I got to witness the beginning of a generation...,5120,Parenting,https://www.reddit.com/r/Parenting/comments/gd...,161,My 4 yr old daughter just called out my toxic ...,[I love this. I make it a point to apologize t...


In [136]:
#cleaning comments
def clean_comments(text):
    '''getting rid of the \n in the comments'''
    for comments in text:
        comments = re.sub('\n','', comments)
    return text

cleaning = lambda x: clean_comments(x)

parenting_posts['comments'] = parenting_posts.comments.apply(cleaning)
parenting_posts.head()

,id,title,score,subreddit,url,num_comments,text,comments
0,76x0q4,[Update]35 year old Dad diagnosed with a termi...,9463,Parenting,https://www.reddit.com/r/Parenting/comments/76...,304,I want you all to know that I had no idea Scot...,[This is beautiful. What an incredible father....
1,gh7f74,Gave birth in our driveway this morning,6236,Parenting,https://www.reddit.com/r/Parenting/comments/gh...,301,Just a sigh of relief via text! We were all pr...,"[Holy moly, what a way to celebrate Mother's D..."
2,bmjbfv,I'm dealing with depression and I always try t...,5632,Parenting,https://www.reddit.com/r/Parenting/comments/bm...,258,Had a rough morning this morning. Dealing with...,"[You know, he learned to be that kind, gentle,..."
3,e03s5e,I’ve never been so proud of my 16 year-old son,5306,Parenting,https://www.reddit.com/r/Parenting/comments/e0...,190,"About 10 years ago, my first wife and my daugh...","[Yeah, I'm crying now too., Beautiful. Kids ar..."
4,gdns9x,I got to witness the beginning of a generation...,5120,Parenting,https://www.reddit.com/r/Parenting/comments/gd...,161,My 4 yr old daughter just called out my toxic ...,[I love this. I make it a point to apologize t...


In [137]:
#vectorizing reddit post titles to tag them with topics
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(parenting_posts.title)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = parenting_posts.id
data_dtm.head()

,1000s,100k,12,16,2017,24,30,35,arrived,attempts,...,trying,update,ve,wife,witness,work,worry,worth,year,years
id,,,,,,,,,,,,,,,,,,,,,
76x0q4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,2,0
gh7f74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bmjbfv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
e03s5e,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
gdns9x,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [138]:
#transposing document-term matrix for topic modeling
tdm = data_dtm.transpose()
tdm.head()

id,76x0q4,gh7f74,bmjbfv,e03s5e,gdns9x,2yi6xu,5l7dbd,5qu7mk,5hjc6u,5pev3i
1000s,0,0,0,0,0,0,0,1,0,0
100k,0,0,0,0,0,0,0,1,0,0
12,0,0,0,0,0,0,0,0,1,0
16,0,0,0,1,0,0,0,0,0,0
2017,0,0,0,0,0,0,0,1,0,0


In [139]:
#convert to gensim format
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v,k) for k,v in cv.vocabulary_.items())

In [140]:
#LDA Topic Modeling for all words
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes = 80)
lda.print_topics()

[(0,
  '0.062*"hide" + 0.034*"proud" + 0.034*"old" + 0.034*"year" + 0.034*"son" + 0.034*"try" + 0.034*"couldn" + 0.034*"today" + 0.034*"dealing" + 0.034*"reaction"'),
 (1,
  '0.055*"year" + 0.055*"old" + 0.030*"years" + 0.030*"update" + 0.030*"terminal" + 0.030*"girl" + 0.030*"tell" + 0.030*"diagnosed" + 0.030*"little" + 0.030*"dad"'),
 (2,
  '0.056*"year" + 0.056*"friend" + 0.056*"explained" + 0.056*"neutrality" + 0.056*"net" + 0.009*"counting" + 0.009*"years" + 0.009*"old" + 0.009*"proud" + 0.009*"morning"'),
 (3,
  '0.083*"like" + 0.045*"daughter" + 0.045*"mimics" + 0.045*"getting" + 0.045*"wife" + 0.045*"pics" + 0.045*"work" + 0.045*"love" + 0.045*"parrot" + 0.008*"counting"'),
 (4,
  '0.040*"son" + 0.040*"years" + 0.022*"loss" + 0.022*"months" + 0.022*"injections" + 0.022*"iui" + 0.022*"100k" + 0.022*"labor" + 0.022*"late" + 0.022*"lincoln"')]

In [141]:
#LDA Topic Modeling for nouns and adjectives only
from nltk import word_tokenize, pos_tag

def nouns_adj(text):
    '''given the title, tokenize the text and pull out only the nouns'''
    tokenized = word_tokenize(text)
    all_nouns_adj = [word for (word, pos) in pos_tag(tokenized) if pos == 'NN' or pos == 'JJ']
    return ' '.join(all_nouns_adj)

nouns_adj_only = pd.DataFrame(parenting_posts.title.apply(nouns_adj))
nouns_adj_only.head()

,title
0,[ ] year old terminal illness year old little ...
1,birth driveway morning
2,depression year old son Today proud reaction
3,ve proud year-old son
4,beginning generational pattern


In [142]:
#vectorize noun_adj words 
cvn = CountVectorizer(stop_words = 'english', max_df=.8)
data_cvn = cvn.fit_transform(nouns_adj_only.title)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = parenting_posts.id
data_dtmn.head()

,baby,beginning,birth,chemical,counting,daughter,depression,driveway,emergency,explained,...,son,stress,term,terminal,today,ve,wife,work,worry,year
id,,,,,,,,,,,,,,,,,,,,,
76x0q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
gh7f74,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
bmjbfv,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
e03s5e,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
gdns9x,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
# Create the gensim parameteres
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [154]:
#printing the topics
ldan = models.LdaModel(corpus=corpusn, num_topics=5, id2word=id2wordn, passes=80)
ldan.print_topics()

[(0,
  '0.132*"year" + 0.101*"old" + 0.069*"daughter" + 0.038*"little" + 0.038*"illness" + 0.038*"girl" + 0.038*"terminal" + 0.038*"explained" + 0.038*"friend" + 0.038*"net"'),
 (1,
  '0.058*"son" + 0.058*"emergency" + 0.058*"loss" + 0.058*"labor" + 0.058*"pregnancy" + 0.058*"stress" + 0.058*"baby" + 0.058*"late" + 0.058*"term" + 0.058*"chemical"'),
 (2,
  '0.026*"counting" + 0.026*"perfect" + 0.026*"daughter" + 0.026*"son" + 0.026*"proud" + 0.026*"wife" + 0.026*"parrot" + 0.026*"work" + 0.026*"pattern" + 0.026*"birth"'),
 (3,
  '0.026*"counting" + 0.026*"perfect" + 0.026*"daughter" + 0.026*"son" + 0.026*"proud" + 0.026*"wife" + 0.026*"parrot" + 0.026*"work" + 0.026*"pattern" + 0.026*"birth"'),
 (4,
  '0.058*"proud" + 0.058*"son" + 0.058*"depression" + 0.058*"reaction" + 0.058*"today" + 0.058*"pattern" + 0.058*"beginning" + 0.058*"driveway" + 0.058*"birth" + 0.058*"morning"')]

In [194]:
#finding the topics for each post and adding it to the table
topic_dictionary = {0:'daughter scared', 1:'stress and emergencies', 
                    2: 'proud moments', 3:'birth', 4:'depression and driving' }
corpus_transformed = ldan[corpusn]
topics_list = [] 
top_num = []
for topic_breakdown in corpus_transformed:
    prevalent_topic = max(topic_breakdown,key=lambda tup:tup[1])[0]
    top_num.append(prevalent_topic)
    topics_list.append(topic_dictionary[prevalent_topic])
topics_list
parenting_posts['problem_tag'] = topics_list
parenting_posts['topic_num'] = top_num

In [195]:
#final table with tags
parenting_posts.head()

,id,title,score,subreddit,url,num_comments,text,comments,problem_tag,topic_num
0,76x0q4,[Update]35 year old Dad diagnosed with a termi...,9463,Parenting,https://www.reddit.com/r/Parenting/comments/76...,304,I want you all to know that I had no idea Scot...,[This is beautiful. What an incredible father....,daughter scared,0
1,gh7f74,Gave birth in our driveway this morning,6236,Parenting,https://www.reddit.com/r/Parenting/comments/gh...,301,Just a sigh of relief via text! We were all pr...,"[Holy moly, what a way to celebrate Mother's D...",depression and driving,4
2,bmjbfv,I'm dealing with depression and I always try t...,5632,Parenting,https://www.reddit.com/r/Parenting/comments/bm...,258,Had a rough morning this morning. Dealing with...,"[You know, he learned to be that kind, gentle,...",depression and driving,4
3,e03s5e,I’ve never been so proud of my 16 year-old son,5306,Parenting,https://www.reddit.com/r/Parenting/comments/e0...,190,"About 10 years ago, my first wife and my daugh...","[Yeah, I'm crying now too., Beautiful. Kids ar...",daughter scared,0
4,gdns9x,I got to witness the beginning of a generation...,5120,Parenting,https://www.reddit.com/r/Parenting/comments/gd...,161,My 4 yr old daughter just called out my toxic ...,[I love this. I make it a point to apologize t...,depression and driving,4


In [ ]:
parenting_posts.to_csv('reddit_parenting_posts.csv')